by Akinde Kadjo

**The goal** here is to get the data into a concise form. Due to its large size (20 GB), the data won't be added to GitHub, but the original data set can be found [here.](https://www.kaggle.com/datasets/drscarlat/driams)

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import os
from glob import glob
from tqdm import tqdm
from tqdm.keras import TqdmCallback
import time
from difflib import SequenceMatcher

# Data Extraction

In [2]:
id2015 = pd.read_csv('id/2015/2015_clean.csv', low_memory=False)
print(id2015.shape)
id2016 = pd.read_csv('id/2016/2016_clean.csv', low_memory=False)
print(id2016.shape)
id2017 = pd.read_csv('id/2017/2017_clean.csv', low_memory=False)
print(id2017.shape)
id2018 = pd.read_csv('id/2018/2018_clean.csv', low_memory=False)
print(id2018.shape)

(3198, 81)
(34868, 90)
(43122, 90)
(30069, 87)


In [3]:
# concatenating them
id_df = pd.concat([id2015, id2016, id2017, id2018], ignore_index=True)
id_df.shape

(111257, 92)

In [4]:
id_df.head(2)

,code,species,laboratory_species,Piperacillin-Tazobactam,Meropenem,Ciprofloxacin,Cefepime,Cotrimoxazole,Ceftazidime,Amikacin,...,Penicillin_without_meningitis,Cefuroxime.1,Ceftazidime-Avibactam,Ceftolozane-Tazobactam,Ampicillin-Sulbactam,Ceftobiprole,Strepomycin_high_level,Isavuconazole,Unnamed: 0.1,Unnamed: 0
0,74969164-613a-4455-ac8e-5666ee0dfade,MIX!Streptococcus pneumoniae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e9adf43d-679b-497c-9849-1fa214838dd3,Staphylococcus epidermidis,Staphylococcus epidermidis,R,R,R,R,S,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Cleaning

In [5]:
#replace "-" with np.nan
id_df.replace({'-':np.nan}, inplace = True)
id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111257 entries, 0 to 111256
Data columns (total 92 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   code                                           111257 non-null  object 
 1   species                                        111257 non-null  object 
 2   laboratory_species                             83653 non-null   object 
 3   Piperacillin-Tazobactam                        28802 non-null   object 
 4   Meropenem                                      29791 non-null   object 
 5   Ciprofloxacin                                  30920 non-null   object 
 6   Cefepime                                       28863 non-null   object 
 7   Cotrimoxazole                                  27042 non-null   object 
 8   Ceftazidime                                    17688 non-null   object 
 9   Amikacin                             

In [6]:
#drop columns with all missing values (0 non-null)
id_df.dropna(axis = 1, how = 'all', inplace = True)
#drop the non-needed columns
id_df.drop(columns =['laboratory_species', 'Unnamed: 0.1', 'Unnamed: 0' ], 
           inplace = True)
id_df.shape

(111257, 84)

In [7]:
#overview of entered values
x = id_df.drop(columns =['code', 'species'])
x.apply(pd.Series.value_counts)

,Piperacillin-Tazobactam,Meropenem,Ciprofloxacin,Cefepime,Cotrimoxazole,Ceftazidime,Amikacin,Levofloxacin,Imipenem,Tobramycin,...,Teicoplanin_GRD,Ceftarolin,Penicillin_without_meningitis,Cefuroxime.1,Ceftazidime-Avibactam,Ceftolozane-Tazobactam,Ampicillin-Sulbactam,Ceftobiprole,Strepomycin_high_level,Isavuconazole
I,403,314.0,711,960,27.0,567.0,759.0,404.0,1160.0,56.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
"I(1), S(1)",36,52.0,77,53,11.0,82.0,175.0,61.0,66.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R,6143,4831.0,6751,5558,4845.0,1888.0,216.0,3569.0,5712.0,1639.0,...,2.0,10.0,NaN,NaN,15.0,35.0,11.0,38.0,NaN,1.0
"R(1), I(1)",19,32.0,38,92,NaN,78.0,12.0,48.0,37.0,7.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
"R(1), I(1), S(1)",1,NaN,7,2,NaN,NaN,NaN,4.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"R(1), S(1)",344,170.0,247,234,391.0,132.0,20.0,114.0,198.0,85.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
R(2),2,2.0,4,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S,21852,24386.0,23081,21958,21768.0,14937.0,16247.0,16811.0,22519.0,16495.0,...,10.0,27.0,4.0,3.0,56.0,72.0,3.0,24.0,4.0,4.0
S(2),2,4.0,4,4,NaN,4.0,4.0,4.0,4.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Correct 'species' column

In [8]:
#checking for inconcistencies
id_df['species'] = id_df['species'].str.lower()
id_df.sort_values('species', inplace = True, ignore_index = True)
id_df['species'].value_counts()

staphylococcus epidermidis              9387
escherichia coli                        7320
staphylococcus aureus                   6994
pseudomonas aeruginosa                  4852
gardnerella vaginalis                   4447
                                        ... 
mix!comamonas kerstersii                   1
mix!corynebacterium flavescens             1
mix!corynebacterium freneyi                1
mix!corynebacterium glucuronolyticum       1
mix!clostridium hathewayi                  1
Name: species, Length: 1227, dtype: int64

There should be about ~800 unique species, the above value of 1227 indicates that there could be mispelling.

In [9]:
#removing'mix!' prefix
id_df['species'] = id_df['species'].str.replace('mix!','')#({'mix!':''}, inplace = True)
id_df.sort_values('species', inplace = True, ignore_index = True)
id_df['species'].value_counts()

staphylococcus epidermidis     9478
escherichia coli               7381
staphylococcus aureus          7065
pseudomonas aeruginosa         4908
gardnerella vaginalis          4620
                               ... 
staphylococcus kloosii            1
clostridium bartlettii            1
aureimonas altamirensis           1
bifidobacterium catenulatum       1
eubacterium sp[2]                 1
Name: species, Length: 840, dtype: int64

We went from 1227 to 840. Now let's take care of slightly misspelled species

In [10]:
#using sequence matcher to match slightly misspelled species name
(r1a,r1b) = ((id_df['species'][0]),(id_df['species'][10]))
r1 = SequenceMatcher(None, r1a, r1b ).ratio()
print(f'{r1a} and {r1b} have a match ratio of {r1}')
(r2a,r2b) = ((id_df['species'][30]),(id_df['species'][60]))
r2 = SequenceMatcher(None, r2a, r2b ).ratio()
print(f'{r2a} and {r2b} have a match ratio of {r2}')

abiotrophia defectiva and abiotrophia defectiva have a match ratio of 1.0
achromobacter insolitus and achromobacter piechaudii have a match ratio of 0.6808510638297872


In [11]:
#setting words as the "same" for words with match ratio higher than 0.9
for i in range(1,len(id_df)):
    (ra,rb) = ((id_df['species'][i-1]),(id_df['species'][i]))
    if SequenceMatcher(None, ra, rb ).ratio()>= 0.9:
        id_df.loc[i,'species'] = id_df['species'][i-1]

id_df['species'].value_counts()        

staphylococcus epidermidis     9478
escherichia coli               7381
staphylococcus aureus          7065
pseudomonas aeruginosa         4908
gardnerella vaginalis          4620
                               ... 
lactobacillus paraplantarum       1
clostridium sp                    1
comamonas aquatica                1
comamonas terrigena               1
desulfovibrio desulfuricans       1
Name: species, Length: 825, dtype: int64

# Data Saving

In [12]:
id_df.to_csv('labels.csv.gz', index=False, compression='gzip')